In [2]:
"This notebook standardizes all the models used for analytical field models"

'This notebook standardizes all the models used for analytical field models'

In [2]:
#Import needed libraries
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve
import math as math
import pandas as pd
from pylab import cm
from matplotlib import colors as colors

C:\Users\craab\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#Basic Functions 
c = 1.97
def Get_i(arr,val):
    """
    Gives index of array closest to desired value
    
    Input:  arr - numpy array
            val - float
    Output: i   - index of closest value to desired float (int)
    """    
    
    i=np.argmin(np.abs(arr-val))
    return i

def real(array):
    re = []
    for i in array:
        re.append(i.real)
    return re


def imag(array):
    im= []
    for i in array:
        im.append(i.imag)
    return im
        
    
def drude(omega, p, gamma, h):
    """
    Drude dielectric
    Input: 
        omega- frequency/energy (eV)
        p- plasma frequency(eV) 
        gamma- damping constant (eV)
        h- background dielectric (epsilon infinity)
    Output:
    Drude dielectric
    """
    z = h - (p**2 * (1/omega) * 1/(omega + (1j) * gamma))
    return z

def lorentz(omega, f, b, gamma):
    """
    Lorentz dielectric
    Input: 
        omega- frequency (eV)
        b- binding frequency(eV) 
        gamma- damping constant (eV)
    Output:
    Lorentz dielectric
    """
    z = f/(b**2 - omega**2 - (1j) * omega*gamma)
    return z

def drude_lorentz(omega, p, gamma1, h, f, b, gamma2):
    """
    Drude Lorentz dielectric 
    Input: 
        omega- frequency (eV)
        b- binding frequency(eV) 
        gamma1- Drude damping constant (eV)
        gamma2- Lorentz damping constant (eV)
        h1- Drude background d|ielectric 
       
    Output:
    Drude-Lorentz dielectric
    """
    z = drude(omega, p, gamma1, h) + lorentz(omega, f, b, gamma2)
    return z

def vacuum(omega):
    """
    Dielectric of vacuum
    """
    vac = np.ones_like(omega)
    return vac

def linear_dielectric(omega, n):
    """
    Dielectric of linear medium with refractive index n
    """
    vac = n**2 * np.ones_like(omega)
    return vac

def semi_infinite(omega, dielectric1, dielectric2):
    """
    Semi-infinite dispersion
    """
    k = (omega/c)*np.emath.sqrt(dielectric1 * dielectric2/(dielectric1 + dielectric2))
    return k


def anisotropic_semi_infinite(omega, dielectric1, dielectric_xy, dielectric_z):
    """
    Anisotropic Semi-Infinite Dispersion
    """
    c = 1.97
    z = (omega/c) * np.emath.sqrt(dielectric1 * dielectric_z) * np.emath.sqrt((dielectric1-dielectric_xy)/
                                                                              (dielectric1**2-dielectric_xy*dielectric_z))
    return z

In [4]:
#Reflection Coefficient Models. Plot the complex reflection coefficient as a function of wavevector and energy


def reflection_coefficient_sf(omega, k, dielectric1, dielectric_xy, dielectric_z):
    """
    Reflection coefficient for single interface
    omega - arrray of energy (E)
    k - array of momentum (K)
    dielectric1 - upper medium 1 dielectric
    dielectric_xy - in-plane lower layer dielectric
    dielectric_z - out-of-plane lower layer dielectric
    """
    im_rp = np.zeros((len(omega), len(k)))
    for i in k:
        for w in omega:
            ind_w = Get_i(omega, w)
            ind_k = Get_i(k, i)
            eps1 = dielectric1[ind_w]
            eps_xy = dielectric_xy[ind_w]
            eps_z = dielectric_z[ind_w]
            k1z = np.emath.sqrt(eps1*(w/c)**2 - i**2)
            if k1z.imag < 0:  #enforce sign conventions for square root
                k1z = -1 *k1z
            if k1z.imag == 0:
                k1z = np.abs(k1z)
            k2z = np.emath.sqrt(eps_xy*(w/c)**2 - (eps_xy/eps_z)*i**2)
            if k2z.imag < 0:       #enforce sign conventions for square root
                k2z = -1 * k2z
            
            r_12 = (eps1*k2z - eps_xy*k1z)/(eps1*k2z + eps_xy*k1z)  #reflection coefficient
            im_rp[ind_w][ind_k] = np.abs(r_12.imag)  #Append absolute val of imaginary part of r_12
    return im_rp



def reflection_coefficient(omega, k, d, dielectric1, dielectric_xy, dielectric_z, dielectric3):
    """
    Standard reflection coefficient for 3-layer models
    omega - arrray of energy (E)
    k - array of momentum (K)
    d- thickness of medium 2
    dielectric1 - upper (medium 1) dielectric
    dielectric_xy - in-plane medium 2 dielectric
    dielectric_z - out-of-plane medium 2 dielectric
    dielectric3 - lower (medium 3) dielectric
    """
    im_rp = np.zeros((len(omega), len(k)))
    for i in k:
        for w in omega:
            ind_w = Get_i(omega, w)
            ind_k = Get_i(k, i)
            eps1 = dielectric1[ind_w]
            eps_xy = dielectric_xy[ind_w]
            eps_z = dielectric_z[ind_w]
            eps3 = dielectric3[ind_w]
            k1z = np.emath.sqrt(eps1*(w/c)**2 - i**2)
            if k1z.imag < 0:
                k1z = -1 *k1z
            if k1z.imag == 0:
                k1z = np.abs(k1z)
            k2z = np.emath.sqrt(eps_xy*(w/c)**2 - (eps_xy/eps_z)*i**2) 
            #enforcing sign convention for k_2z does nothing
    
            k3z = np.emath.sqrt(eps3*(w/c)**2 - i**2)
            if k3z.imag < 0:
                k3z = -1 * k3z
            if k3z.imag == 0:
                k3z = np.abs(k3z)
            r_12 = (eps1*k2z - eps_xy*k1z)/(eps1*k2z + eps_xy*k1z)
            r_23 = (eps_xy*k3z - eps3*k2z)/(eps_xy*k3z + eps3 * k2z)
            rp = (r_12 + r_23*np.exp(2*(1j)*k2z*d))/(1 + r_12*r_23 * np.exp(2*(1j)*k2z*d))
            im_rp[ind_w][ind_k] = np.abs(rp.imag)
    return im_rp


def reflection_coefficient_4layer(omega, k, d1, d2, dielectric1, 
                           dielectric_xy, 
                                  dielectric_z, dielectric3, dielectric4):
    """
    Reflection coefficient for 4-layer model
    omega - arrray of energy (E)
    d1 - thickness of medium 2
    d2 - thickness of medium 3
    k - array of momentum (K)
    d- thickness of medium 2
    dielectric1 - upper (medium 1) dielectric
    dielectric_xy - in-plane medium 2 dielectric
    dielectric_z - out-of-plane medium 2 dielectric
    dielectric3 - medium 3 dielectric
    dielectric4 - bottom (medium 4) dielectric
    """
    c = 1.97
    im_rp = np.zeros((len(omega), len(k)))
    for i in k:
        for w in omega:
            ind_w = Get_i(omega, w)
            ind_k = Get_i(k, i)
            eps1 = dielectric1[ind_w]
            eps_xy = dielectric_xy[ind_w]
            eps_z = dielectric_z[ind_w]
            eps3 = dielectric3[ind_w]
            eps4 = dielectric4[ind_w]
            k1z = np.emath.sqrt(eps1*(w/c)**2 - i**2)
            if k1z.imag < 0:
                k1z = -1 * k1z
            k2z = np.emath.sqrt(eps_xy*(w/c)**2 - (eps_xy/eps_z)*i**2)
            if k2z.imag < 0:
                k2z = -1 * k2z
            k3z = np.emath.sqrt(eps3*(w/c)**2 - i**2)
            if k3z.imag < 0:
                k3z = -1 * k3z
            k4z = np.emath.sqrt(eps4*(w/c)**2 - i**2)
            if k4z.imag < 0:
                k4z = -1 * k4z
            r_12 = (eps1*k2z -eps_xy*k1z )/(eps_xy*k1z + eps1 * k2z)
            r_23 = (eps_xy*k3z - eps3 * k2z)/(eps_xy*k3z + eps3 * k2z)
            r_34 = (eps3*k4z - eps4*k3z )/(eps3*k4z + eps4*k3z)
            alpha2 = k2z * d1 * (1j)
            alpha3 = k3z * d2 * (1j)
            rp = (r_12 + r_12*r_23*r_34*np.exp(2*alpha3) + r_23*np.exp(2*alpha2) + r_34 *np.exp(2*alpha2 + 2*alpha3) )/(1 + r_23*r_34 * np.exp(2*alpha3) + r_12*r_23*np.exp(2*alpha2) + r_12*r_34*np.exp(2*alpha2 + 2*alpha3))
            ind_w = Get_i(omega, w)
            ind_k = Get_i(k, i)
            im_rp[ind_w][ind_k] = np.abs(rp.imag)
    return im_rp




def plot_complex_k(omega, k_re, k_im, d, eps1, eps_xy, eps_z, eps3):
    """
    Here we plot Im(r_p) as a function of complex k (Re k, Im k) for one energy value (omega) for 3-layers
    This allows for visualization of poles in complex space
    omega - energy value(float)
    k_re - array of real k_x values
    k_im - array of imag k_x values
    d - thickness
    eps1 - value of dielectric1 (float)
    eps_xy - value of in-plane dielectric at omega(float)
    eps_z - value of out-of-plane dielectric at omega(float)
    eps3 - value of dielectric3 at omega(float)
    """
    im_rp = np.zeros((len(k_im), len(k_re)))
    for i in k_re:
        for j in k_im:
            ind_i = Get_i(k_re, i)
            ind_j = Get_i(k_im, j)
            k = i + (1j)*j
            k1z = np.emath.sqrt(eps1*(omega/c)**2 - k**2)
            if k1z.imag < 0:
                k1z = -1 *k1z
            if k1z.imag == 0:
                k1z = np.abs(k1z)
            k2z = np.emath.sqrt(eps_xy*(omega/c)**2 - (eps_xy/eps_z)*k**2)
            k3z = np.emath.sqrt(eps3*(omega/c)**2 - k**2)
            if k3z.imag < 0:
                k3z = -1 * k3z
            if k3z.imag == 0:
                k3z = np.abs(k3z)
            r_12 = (eps1*k2z - eps_xy*k1z)/(eps1*k2z + eps_xy*k1z)
            r_23 = (eps_xy*k3z - eps3*k2z)/(eps_xy*k3z + eps3 * k2z)
            rp = (r_12 + r_23*np.exp(2*(1j)*k2z*d))/(1 + r_12*r_23 * np.exp(2*(1j)*k2z*d))
            im_rp[ind_j][ind_i] = np.abs(rp.imag)
    return im_rp




def plot_complex_k4(omega, k_re, k_im, d1, d2, eps1, eps_xy, eps_z, eps3, eps4):
    """
    Here we plot Im(r_p) as a function of complex k (Re k, Im k) for one energy value (omega) for 4-layers
    This allows for visualization of poles in complex space
    omega - energy value(float)
    k_re - array of real k_x values
    k_im - array of imag k_x values
    d1 - thickness of medium 2
    d2 - thickness of medium 3
    eps1 - value of dielectric1 (float)
    eps_xy - value of in-plane dielectric at omega (float)
    eps_z - value of out-of-plane dielectric at omega(float)
    eps3 - value of dielectric3 at omega(float)
    eps4- value of dielectric4 at omega(float)
    """
    im_rp = np.zeros((len(k_im), len(k_re)))
    for i in k_re:
        for j in k_im:
            ind_i = Get_i(k_re, i)
            ind_j = Get_i(k_im, j)
            k = i + (1j)*j
            k1z = np.emath.sqrt(eps1*(omega/c)**2 - k**2)
            if k1z.imag < 0:
                k1z = -1 *k1z
            if k1z.imag == 0:
                k1z = np.abs(k1z)
            k2z = np.emath.sqrt(eps_xy*(omega/c)**2 - (eps_xy/eps_z)*k**2)
            if k2z.imag < 0:
                k2z = -1 * k2z
            k3z = np.emath.sqrt(eps3*(omega/c)**2 - k**2)
            if k3z.imag < 0:
                k3z = -1 * k3z
            if k3z.imag == 0:
                k3z = np.abs(k3z)
            k4z = np.emath.sqrt(eps4*(omega/c)**2 - k**2)
            if k4z.imag < 0:
                k4z = -1 * k4z
            if k4z.imag == 0:
                k4z = np.abs(k4z)
            r_12 = (eps1*k2z - eps_xy*k1z)/(eps1*k2z + eps_xy*k1z)
            r_23 = (eps_xy*k3z - eps3*k2z)/(eps_xy*k3z + eps3 * k2z)
            r_34 = (eps3*k4z - eps4*k3z )/(eps3*k4z + eps4*k3z)
            alpha2 = k2z * d1 * (1j)
            alpha3 = k3z * d2 * (1j)
            rp = (r_12 + r_12*r_23*r_34*np.exp(2*alpha3) + r_23*np.exp(2*alpha2) + r_34 *np.exp(2*alpha2 + 2*alpha3) )/(1 + r_23*r_34 * np.exp(2*alpha3) + r_12*r_23*np.exp(2*alpha2) + r_12*r_34*np.exp(2*alpha2 + 2*alpha3))
            im_rp[ind_j][ind_i] = np.abs(rp.imag)
    return im_rp

def plot_rp(rp, E, k, vmin0 = 0, vmax0 = None):
    """
    Plot any reflection coefficient as heatmap
    rp - reflection coefficient 
    E - energy array
    k - momentum array
    vmin0 - lower bound of colorbar
    vmax0 - upper bound of colorbar
    """
    f, a = plt.subplots(figsize = ((10, 6)), dpi = 600)
    im = a.imshow(rp, cmap = cm.plasma, aspect='auto'
           ,extent=[k[0],k[-1],E[0],E[-1]],vmin= vmin0,vmax= vmax0, origin = 'lower')
    cbar = plt.colorbar(im, pad = 0.01)
    cbar.set_label('Im $r_p$' )
    a.set_xlim(k[0],k[-1])
    a.set_ylim(E[0], E[-1])
    a.set_ylabel('Energy (eV)')
    a.set_xlabel('Re($k_x$) 10$^5$ cm$^{-1}$')
    plt.plot(E/1.97, E, color = 'white', linestyle = 'dashdot')

In [5]:
# Field solvers. For calculating the values of fields from the transfer matrix method

def solve_point_disp(omega, d, eps1, eps_xy, eps_z, eps3, k_min, k_max, k_im_min = 0, k_im_max = 5):
    """
    Solver for finding the zeros of the 3-layer dispersion relation. 
    This gives k_x from constant energy
    omega - energy value(float)
    d - thickness
    eps1 - value of dielectric1 (float)
    eps_xy - value of in-plane dielectric at omega(float)
    eps_z - value of out-of-plane dielectric at omega(float)
    eps3 - value of dielectric3 at omega(float) 
    k_min - lower bound for solutions in units of k/k0
    k_max - upper bound for solutions in units of k/k0
    k_im_min - lower bound (actual value) of im(k_x)
    k_im_max - upper bound (actual value) of im(k_x)
    """
    def disp(k, omega):  #function to minimize to finds zeros
        k = k[0] + (1j)*k[1]  #writing complex k as array so fsolve can solve 2 simultaneous equations
        k1z = np.emath.sqrt(eps1*(omega/c)**2 - k**2)
        k1z = np.emath.sqrt(eps1*(omega/c)**2 - k**2)
        k2z = np.emath.sqrt(eps_xy*(omega/c)**2 - (eps_xy/eps_z)*k**2)
        k3z = np.emath.sqrt(eps3*(omega/c)**2 - k**2)
        if k1z.imag < 0:   #Imaginary k_1z positive
            k1z = -1 * k1z
        if k1z.imag == 0:
            k1z = np.abs(k1z)
        if k2z.imag < 0:  #imaginary k_2z positive
            k2z = -1 * k2z
        if k2z.imag == 0:
            k2z = np.abs(k2z)
        if k3z.imag < 0: #Imaginary k_3z positive
            k3z = -1 * k3z
                #print("k3z sign flip")
        if k3z.imag == 0:
            k3z = np.abs(k3z)
        r_12 = (eps1*k2z - eps_xy*k1z)/(eps_xy*k1z + eps1 * k2z)
        r_23 = (eps_xy*k3z- eps3*k2z)/(eps_xy*k3z + eps3 * k2z)
        disp = 1 + r_12*r_23 * np.exp(2*(1j)*k2z*d)
        return np.abs(disp.real), np.abs(disp.imag)
    k_vals = []
    disp_vals = []
    
    #Need to find good guess values since fsolve is very dependent on this
    #iterate through k-space trying to find roughly best starting values
    k_re = np.linspace(k_min*omega/c, k_max*omega/c, 150)
    k_im = np.linspace(k_im_min, k_im_max, 150)  #Iterate through values of real, imag k to find estimate of k
    for i in k_re:
        for j in k_im:
            k_vals.append(i + (1j) * j)
            disp_vals.append(disp([i, j], omega)[0]**2 + disp([i, j], omega)[1]**2)
    best_ind = np.nanargmin(disp_vals)  #append values of k_re, k_im value that maximizes r_p
    best_k = k_vals[best_ind]  #Estimated k value to use as initial guess
    
    #Second round of iteration to make the initial guess more precise
    if best_k.real == k_min: range1 = -0.1* best_k.real  #Unique case in which solution is at limits
    if best_k.real == k_max: range2 = -0.1 * best_k.real
    else:    
        range1 = 0.1 * (best_k.real - (k_min*omega/c))  #search for values just to left and right of first guess
        range2 =  0.1 * ((k_max*omega/c) - best_k.real)
    k_re2 = np.linspace(best_k.real - range1, best_k.real + range2, 200)
    k_im2 = np.linspace(best_k.imag * 0.9,best_k.imag * 1.1, 150)
    k_vals2 = []
    disp_vals2 = []
    for i in k_re2:
        for j in k_im2:
            k_vals2.append(i + (1j) * j)
            disp_vals2.append(disp([i, j], omega)[0]**2 + disp([i, j], omega)[1]**2)
            
    best_ind2 = np.nanargmin(disp_vals2)  #append values of k_re, k_im value that maximizes r_p
    best_k2 = k_vals2[best_ind2]  #Initual guess value from 2nd round of iteration
    
    #Now we actually apply fsolve
    test1 = disp([best_k2.real, best_k2.imag], omega)[0]**2 + disp([best_k2.real, best_k2.imag], omega)[1]**2  #show quality of initial guess soln 
    root = fsolve(disp, [best_k2.real, best_k2.imag], args = (omega))  #fsolve 2 simultaneous eqns
    best_k3 = root[0] + 1j * root[1]  #fsolve solution
    test2 = disp([best_k3.real, best_k3.imag], omega)[0]**2 + disp([best_k3.real, best_k3.imag], omega)[1]**2 #quality of fsolve soln
    
    #Backup if fsolve fails for some reason: compares how close initial guess, fsolve roots are to zero
    # picks which root is closer to zero
    if test2 < test1 and best_k3.real >= (k_min * omega/c) and best_k3.real <= (k_max * omega/c) and best_k3.imag >= 0: 
        print("used fsolve")
        return best_k3
    else: 
        print("used manual")  #if this reads out that is not a good sign
        
        return best_k2

def solve_point_disp4(omega, d1,d2, eps1, eps_xy, eps_z, eps3, eps4, k_min, k_max, k_im_min = 0, 
                      k_im_max = 5):
    """
    Solver for finding the zeros of the 3-layer dispersion relation. 
    This gives k_x from constant energy
    omega - energy value(float)
    d1 - thickness of medium 2
    d2 - thickness of medium 3
    eps1 - value of dielectric1 (float)
    eps_xy - value of in-plane dielectric at omega(float)
    eps_z - value of out-of-plane dielectric at omega(float)
    eps3 - value of dielectric3 at omega(float)
    eps4 - value of dielectric4 at omega(float)
    k_min - lower bound for solutions in units of k/k0
    k_max - upper bound for solutions in units of k/k0
    k_im_min - lower bound (actual value) of im(k_x)
    k_im_max - upper bound (actual value) of im(k_x)
    """
    def disp(k, omega):
        k = k[0] + (1j)*k[1]
        k1z = np.emath.sqrt(eps1*(omega/c)**2 - k**2)
        k1z = np.emath.sqrt(eps1*(omega/c)**2 - k**2)
        k2z = np.emath.sqrt(eps_xy*(omega/c)**2 - (eps_xy/eps_z)*k**2)
        k3z = np.emath.sqrt(eps3*(omega/c)**2 - k**2)
        if k1z.imag < 0:   #Imaginary k_1z positive
            k1z = -1 * k1z
        if k1z.imag == 0:
            k1z = np.abs(k1z)
        if k2z.imag < 0:  #imaginary k_2z positive
            k2z = -1 * k2z
        if k2z.imag == 0:
            k2z = np.abs(k2z)
        if k3z.imag < 0: #Imaginary k_3z positive
            k3z = -1 * k3z
                #print("k3z sign flip")
        if k3z.imag == 0:
            k3z = np.abs(k3z)
        k4z = np.emath.sqrt(eps4*(omega/c)**2 - k**2)
        if k4z.imag < 0:
            k4z = -1 * k4z
        r_12 = (eps1*k2z - eps_xy*k1z)/(eps_xy*k1z + eps1 * k2z)
        r_23 = (eps_xy*k3z- eps3*k2z)/(eps_xy*k3z + eps3 * k2z)
        r_34 = (eps3*k4z - eps4*k3z )/(eps3*k4z + eps4*k3z)
        alpha2 = k2z * d1 * (1j)
        alpha3 = k3z * d2 * (1j)
        disp = 1 + r_23*r_34 * np.exp(2*alpha3) + r_12*r_23*np.exp(2*alpha2) + r_12*r_34*np.exp(2*alpha2 + 2*alpha3)
        return np.abs(disp.real), np.abs(disp.imag)
    k_vals = []
    disp_vals = []
    
    #Need to find good guess values since fsolve is very dependent on this
    #iterate through k-space trying to find roughly best starting values
    k_re = np.linspace(k_min*omega/c, k_max*omega/c, 150)
    k_im = np.linspace(k_im_min, k_im_max, 150)  #Iterate through values of real, imag k to find estimate of k
    for i in k_re:
        for j in k_im:
            k_vals.append(i + (1j) * j)
            disp_vals.append(disp([i, j], omega)[0]**2 + disp([i, j], omega)[1]**2)
    best_ind = np.nanargmin(disp_vals)  #append values of k_re, k_im value that maximizes r_p
    best_k = k_vals[best_ind]  #Estimated k value to use as initial guess
    
    #Second round of iteration to make the initial guess more precise
    if best_k.real == k_min: range1 = -0.1* best_k.real  #Unique case in which solution is at limits
    if best_k.real == k_max: range2 = -0.1 * best_k.real
    else:    
        range1 = 0.1 * (best_k.real - (k_min*omega/c))  #search for values just to left and right of first guess
        range2 =  0.1 * ((k_max*omega/c) - best_k.real)
    k_re2 = np.linspace(best_k.real - range1, best_k.real + range2, 200)
    k_im2 = np.linspace(best_k.imag * 0.9,best_k.imag * 1.1, 150)
    k_vals2 = []
    disp_vals2 = []
    for i in k_re2:
        for j in k_im2:
            k_vals2.append(i + (1j) * j)
            disp_vals2.append(disp([i, j], omega)[0]**2 + disp([i, j], omega)[1]**2)
            
    best_ind2 = np.nanargmin(disp_vals2)  #append values of k_re, k_im value that maximizes r_p
    best_k2 = k_vals2[best_ind2]  #Initual guess value from 2nd round of iteration
    
    #Now we actually apply fsolve
    test1 = disp([best_k2.real, best_k2.imag], omega)[0]**2 + disp([best_k2.real, best_k2.imag], omega)[1]**2  #show quality of initial guess soln 
    root = fsolve(disp, [best_k2.real, best_k2.imag], args = (omega))  #fsolve 2 simultaneous eqns
    best_k3 = root[0] + 1j * root[1]  #fsolve solution
    test2 = disp([best_k3.real, best_k3.imag], omega)[0]**2 + disp([best_k3.real, best_k3.imag], omega)[1]**2 #quality of fsolve soln
    
    #Backup if fsolve fails for some reason: compares how close initial guess, fsolve roots are to zero
    # picks which root is closer to zero
    if test2 < test1 and best_k3.real >= (k_min * omega/c) and best_k3.real <= (k_max * omega/c) and best_k3.imag >= 0: 
        print("used fsolve")
        return best_k3
    else: 
        print("used manual")  #if this reads out that is not a good sign
        
        return best_k2

In [1]:
#Calculate values of fields from Maxwell's equations
def calculate_fields_sf(omega, k_x, eps1, eps_xy, eps_z,x_array, z_array):
    """
    Calculate electrmagnetic fields for semi-infinite interface
    omega - energy value(float)
    k_x - solution to 10^5 cm^-1
    d - thickness
    eps1 - value of dielectric1 (float)
    eps_xy - value of in-plane dielectric at omega(float)
    eps_z - value of out-of-plane dielectric at omega(float)
    
    """
    x = x_array
    z = z_array
    k_1z = np.emath.sqrt(eps1*(omega/c)**2 - k_x**2)
    k_2z = np.emath.sqrt(eps_xy*(omega/c)**2 - (eps_xy/eps_z)*k_x**2)
    if (k_1z.imag) < 0:
        k_1z = -1 * k_1z
    if (k_2z.imag) < 0:
        k_2z = -1 * k_2z
    print("k_1z " + str(k_1z))
    print("k_2z " + str(k_2z))
    
    E_x_re = np.zeros((len(z), len(x)))
    E_x_im = np.zeros((len(z), len(x)))
    
    E_z_re = np.zeros((len(z), len(x)))
    E_z_im = np.zeros((len(z), len(x)))
    
    Hy_re = np.zeros((len(z), len(x)))
    Hy_im = np.zeros((len(z), len(x)))
    
    S_x = np.zeros((len(z), len(x)))
    S_z = np.zeros((len(z), len(x)))
    
    E0 = 1
    E_1z = (k_x/k_1z)
    E_2z = -(eps_xy/eps_z)*(k_x/k_2z)
    Hy1 = -(omega*eps1/k_1z)
    Hy2 = (omega*eps_xy/k_2z)
    
    for i in range(len(x)):
        for j in range(len(z)):
            if z[j] < 0: #medium 1
                x_value = E0 * np.exp((-1j)*k_1z*z[j]) * np.exp((1j)*k_x*x[i])
                z_value = E_1z * x_value
                h_value = Hy1 * x_value
                sx_value = -0.5*z_value * np.conj(h_value)
                sz_value = 0.5*x_value * np.conj(h_value)
                
            elif z[j] > 0: #medium 2
                x_value = E0 * np.exp((1j)*k_2z*z[j]) * np.exp((1j)*k_x*x[i])
                z_value = E_2z * x_value
                h_value = Hy2 * x_value
                sx_value = -0.5*z_value * np.conj(h_value)
                sz_value = 0.5 * x_value * np.conj(h_value)
                
            E_x_re[j][i] = x_value.real 
            E_x_im[j][i] = x_value.imag
            
            E_z_re[j][i] = z_value.real
            E_z_im[j][i] = z_value.imag
            
            Hy_re[j][i] = h_value.real
            Hy_im[j][i] = h_value.imag
            
            S_x[j][i] = sx_value.real
            S_z[j][i] = sz_value.real
            
        
    E_x = E_x_re + (1j)*E_x_im
    E_z = E_z_re +(1j)*E_z_im
    Hy = Hy_re + (1j)*Hy_im
    return E_x, E_z, Hy, S_x, S_z


def calculate_fields(omega, k_x, d, eps1, eps_xy, eps_z, eps3, x_array, z_array):
    """
    Use transfer matrix to find fields from initial values for 3 layer model
    omega - energy value in eV (float)
    k_x - 10^5 cm^-1 (float)
    d - thickness in 10^-5 cm (float)
    eps1 - value of dielectric1 (float)
    eps_xy - value of in-plane dielectric at omega(float)
    eps_z - value of out-of-plane dielectric at omega(float)
    eps3 - value of dielectric3 (float)
    x_array - x values (array)
    z_array - z values (array)
    """
    x = x_array
    z = z_array
    k_1z = np.emath.sqrt(eps1*(omega/c)**2 - k_x**2)
    k_2z = np.emath.sqrt(eps_xy*(omega/c)**2 - (eps_xy/eps_z)*k_x**2)
    #Define all positive vectors as decaying downwards
    if k_1z.imag < 0:   #Imaginary k_1z positive
        k_1z = -1 * k_1z
        print("k1z sign flip")
    if k_2z.imag < 0:  #imaginary k_2z positive
        k_2z = 1 * k_2z
    k_3z = np.emath.sqrt(eps3*(omega/c)**2 - k_x**2)
    if k_3z.imag < 0: #Imaginary k_3z positive
        k_3z = -1 * k_3z
        print("k3z sign flip")
   
    #Fresnel coefficients
    alpha = (1j) * k_2z * d  #Coefficients 
    r_12 =  (eps1*k_2z - eps_xy*k_1z)/(eps1*k_2z + eps_xy*k_1z)  
    r_23 = (eps_xy*k_3z - eps3*k_2z)/(eps_xy*k_3z + eps3 * k_2z)
    t_23 = 1 + r_23
    t_12 = 1 + r_12
    
    
    #Calculate Fields From Fresnel Coefficients
    E_3x = 1
    E_2x_plus0 = E_3x/t_23
    E_2x_minus0 = E_3x*r_23/t_23
    
    
    E_2x_plus = E_2x_plus0 * np.exp(-alpha)
    E_2x_minus = E_2x_minus0 * np.exp(alpha)
    E_1x = (1/t_12)*(r_12*E_2x_plus + E_2x_minus)

    E_x_re = np.zeros((len(z), len(x)))
    E_x_im = np.zeros((len(z), len(x)))
    
    E_z_re = np.zeros((len(z), len(x)))
    E_z_im = np.zeros((len(z), len(x)))
    
    Hy_re = np.zeros((len(z), len(x)))
    Hy_im = np.zeros((len(z), len(x)))
    
    S_x = np.zeros((len(z), len(x)))
    S_z = np.zeros((len(z), len(x)))
    for i in range(len(x)):
        for j in range(len(z)):
            if z[j] < 0:  #Medium 1
                x_value = E_1x * np.exp((-1j)*k_1z*z[j]) * np.exp((1j)*k_x*x[i])
                z_value = (k_x/k_1z)*E_1x*np.exp((-1j)*k_1z*z[j]) * np.exp((1j)*k_x*x[i])
                h_value = -1*(omega/k_1z) * eps1 * E_1x * np.exp((-1j)*k_1z*z[j]) * np.exp((1j)*k_x*x[i])
                sx_value = z_value * np.conj(h_value)
                sz_value = x_value * np.conj(h_value)
            elif 0 <= z[j] and z[j] <= d: #Medium 2
                x_value = (E_2x_plus0 * np.exp((1j)*k_2z*(z[j]-d)) + E_2x_minus0*np.exp((-1j)*k_2z*(z[j]-d)))* np.exp((1j)*k_x*x[i])
                z_value =  -(eps_xy/eps_z)*(k_x/k_2z)*(E_2x_plus0 *np.exp((1j)*k_2z*(z[j]-d)) - E_2x_minus0*np.exp((-1j)*k_2z*(z[j]-d)) )* np.exp((1j)*k_x*x[i])
                h_value = (omega/k_2z) * eps_xy * (E_2x_plus0 *np.exp((1j)*k_2z*(z[j]-d)) - E_2x_minus0*np.exp((-1j)*k_2z*(z[j]-d)))* np.exp((1j)*k_x*x[i])
                sx_value = z_value * np.conj(h_value)
                sz_value = x_value * np.conj(h_value)
            elif z[j] > d: #Medium 3
                x_value = E_3x * np.exp((1j)*k_3z*(z[j]-d))* np.exp((1j)*k_x*x[i])
                z_value = -(k_x/k_3z)*E_3x*np.exp((1j)*k_3z*(z[j]-d))* np.exp((1j)*k_x*x[i])
                h_value = (omega/k_3z)* eps3 * E_3x * np.exp((1j)*k_3z*(z[j]-d))* np.exp((1j)*k_x*x[i])
                sx_value = z_value * np.conj(h_value)
                sz_value = x_value * np.conj(h_value)
            
            E_x_re[j][i] = x_value.real 
            E_x_im[j][i] = x_value.imag
            
            E_z_re[j][i] = z_value.real
            E_z_im[j][i] = z_value.imag
            
            Hy_re[j][i] = h_value.real
            Hy_im[j][i] = h_value.imag
            
            S_x[j][i] = -0.5 * sx_value.real
            S_z[j][i] = 0.5 * sz_value.real
            
    
    E_x = E_x_re + (1j)*E_x_im
    E_z = E_z_re +(1j)*E_z_im
    Hy = Hy_re + (1j)*Hy_im
    return E_x, E_z, Hy, S_x, S_z


def calculate_fields_4l(omega, k_x, d1,d2, eps1, eps_xy, eps_z, eps3, eps4, x_array, z_array):
    """
    With given energy, k_x use transfer matrix to find the fields 
    omega - energy value(float)
    k_x - solution to 10^5 cm^-1
    d - thickness
    eps1 - value of dielectric1 (float)
    eps_xy - value of in-plane dielectric at omega(float)
    eps_z - value of out-of-plane dielectric at omega(float)
    eps3 - value of dielectric3 (float)
    eps4 - value of dielectric4 (float)
    """
    x = x_array
    z = z_array
    #Define all positive vectors as decaying downwards
    k_1z = np.emath.sqrt(eps1*(omega/c)**2 - k_x**2)
    k_2z = np.emath.sqrt(eps_xy*(omega/c)**2 - (eps_xy/eps_z)*k_x**2)
    if k_1z.imag < 0:   #Imaginary k_1z positive
        k_1z = -1 * k_1z
   
    if k_2z.imag < 0:  #imaginary k_2z positive
        k_2z = 1 * k_2z
    
    k_3z = np.emath.sqrt(eps3*(omega/c)**2 - k_x**2)
    if k_3z.imag < 0: #Imaginary k_3z positive
        k_3z = -1 * k_3z
    
    k_4z = np.emath.sqrt(eps4*(omega/c)**2 - k_x**2)
    if k_4z.imag < 0: #Imaginary k_3z positive
        k_4z = -1 * k_4z
    
    if k_4z.imag == 0:
        k_4z = np.abs(k_4z)
   
    #Fresnel Coefficients
    alpha2 = (1j) * k_2z * d1
    alpha3 = (1j) * k_3z * d2
    r_12 =  (eps1*k_2z - eps_xy*k_1z)/(eps1*k_2z + eps_xy*k_1z)
    r_23 = (eps_xy*k_3z - eps3*k_2z)/(eps_xy*k_3z + eps3 * k_2z)
    r_34 =(eps3*k_4z - eps4*k_3z )/(eps3*k_4z + eps4*k_3z)
    t_23 = 1 + r_23
    t_12 = 1 + r_12
    t_34 = 1 + r_34
    
    #Calculate Fields from transfer matrix method
    E_4x = 1
    E_3x_plus = E_4x/t_34
    E_3x_minus = r_34 * E_4x/t_34
    
    E_2x_plus = (1/t_23) * (E_3x_plus*np.exp(-alpha3) + r_23 * E_3x_minus * np.exp(alpha3))
    E_2x_minus = (1/t_23) * (r_23*E_3x_plus*np.exp(-alpha3) + E_3x_minus * np.exp(alpha3))
    
    E_1x = (1/t_12) * (r_12*E_2x_plus*np.exp(-alpha2) + E_2x_minus * np.exp(alpha2))
    
    

    
    
    E_x_re = np.zeros((len(z), len(x)))
    E_x_im = np.zeros((len(z), len(x)))
    
    E_z_re = np.zeros((len(z), len(x)))
    E_z_im = np.zeros((len(z), len(x)))
    
    Hy_re = np.zeros((len(z), len(x)))
    Hy_im = np.zeros((len(z), len(x)))
    
    S_x = np.zeros((len(z), len(x)))
    S_z = np.zeros((len(z), len(x)))
    for i in range(len(x)):
        for j in range(len(z)):
            if z[j] < 0:  #Medium 1
                x_value = E_1x * np.exp((-1j)*k_1z*z[j]) * np.exp((1j)*k_x*x[i])
                z_value = (k_x/k_1z)*E_1x*np.exp((-1j)*k_1z*z[j]) * np.exp((1j)*k_x*x[i])
                h_value = -(omega/k_1z) * eps1 * E_1x *np.exp((-1j)*k_1z*z[j]) * np.exp((1j)*k_x*x[i])
                sx_value = z_value * np.conj(h_value)
                sz_value = x_value * np.conj(h_value)
            elif 0 <= z[j] and z[j] <= d1: #Medium 2
                x_value = (E_2x_plus * np.exp((1j)*k_2z*(z[j]-d1)) + E_2x_minus*np.exp((-1j)*k_2z*(z[j]-d1)))* np.exp((1j)*k_x*x[i])
                z_value =  -(eps_xy/eps_z)*(k_x/k_2z)*(E_2x_plus *np.exp((1j)*k_2z*(z[j]-d1)) - E_2x_minus*np.exp((-1j)*k_2z*(z[j]-d1)) )* np.exp((1j)*k_x*x[i])
                h_value = (omega/k_2z) * eps_xy * (E_2x_plus *np.exp((1j)*k_2z*(z[j]-d1)) - E_2x_minus*np.exp((-1j)*k_2z*(z[j]-d1)))* np.exp((1j)*k_x*x[i])
                sx_value = z_value * np.conj(h_value)
                sz_value = x_value * np.conj(h_value)
            elif z[j] > d1 and z[j] <= (d1+d2): #Medium 3
                x_value = (E_3x_plus * np.exp((1j)*k_3z*(z[j]-d1- d2)) + E_3x_minus * np.exp((-1j)*k_3z*(z[j]-d1- d2)) )* np.exp((1j)*k_x*x[i])
                z_value = -(k_x/k_3z)*(E_3x_plus * np.exp((1j)*k_3z*(z[j]-d1- d2)) - E_3x_minus * np.exp((-1j)*k_3z*(z[j]-d1- d2)) )* np.exp((1j)*k_x*x[i])
                h_value = (omega/k_3z)*eps3*(E_3x_plus * np.exp((1j)*k_3z*(z[j]-d1- d2)) - E_3x_minus * np.exp((-1j)*k_3z*(z[j]-d1- d2)) )* np.exp((1j)*k_x*x[i])
                sx_value = z_value * np.conj(h_value)
                sz_value = x_value * np.conj(h_value)
            elif z[j] > (d1+d2): #Medium 4
                x_value = E_4x*np.exp((1j)*k_4z*(z[j]-d1 - d2))* np.exp((1j)*k_x*x[i])
                z_value = -(k_x/k_4z)*E_4x* np.exp((1j)*k_4z*(z[j]-d1-d2))* np.exp((1j)*k_x*x[i])
                h_value = (omega/k_4z)* eps4 *E_4x*np.exp((1j)*k_4z*(z[j]-d1-d2))* np.exp((1j)*k_x*x[i])
                sx_value = z_value * np.conj(h_value)
                sz_value = x_value * np.conj(h_value)
            
            E_x_re[j][i] = x_value.real 
            E_x_im[j][i] = x_value.imag
            
            E_z_re[j][i] = z_value.real
            E_z_im[j][i] = z_value.imag
            
            Hy_re[j][i] = h_value.real
            Hy_im[j][i] = h_value.imag
            
            S_x[j][i] = -0.5 * sx_value.real
            S_z[j][i] = 0.5 * sz_value.real
            
    
    E_x = E_x_re + (1j)*E_x_im
    E_z = E_z_re +(1j)*E_z_im
    Hy = Hy_re + (1j)*Hy_im
    return E_x, E_z, Hy, S_x, S_z

In [212]:
#Visualize Fields
def plot_fields_sf(omega, E, dielectric1, dielectric_xy, 
                      dielectric_z, fnameE = None, fnameS = None, spacing = 0.1, 
                   z_lower = 0.5, z_upper = -0.5, x_pixels = 50, z_pixels = 50, 
                   arrow_scaleE = 4, arrow_scaleS = 3, equal_mag = False):
    """
    Function for plotting Semi-infinite fields
    omega - energy value to evalulate (float)
    E - energy array
    dielectric 1 -  medium 1 dielectric (array)
    dielectric_xy - medium 2 dielectric xy (array)
    dielectric_z - medium 2 dielectric zz (array)
    fnameE - file name to save electric field image as
    fnameS - file name to save poynting vector image as
    spacing - how much space between zero line, arrows in arrow plot
    z_lower - spatial limit in +z (which is the bottom of the figure!)
    z_upper - spatial limit in -z (which is the top of the figure!)
    x_pixels -  x pixels for field plot
    z_pixels - z pixels for field plot
    arrow_scaleE - scaling for E field arrows
    arrow_scaleS - scaling for S field arrows
    equal_mag - if true makes all arrows the same length
    
    """
    c = 1.97
    ind = Get_i(omega, E)  #Find ind of desired energy
    omega2 = E[ind]  #assign omega2 to value in E array closest to desired value 
    eps1 = dielectric1[ind]
    eps_xy = dielectric_xy[ind]
    eps_z = dielectric_z[ind]
    k_x = (omega2/c) * np.emath.sqrt(eps_z*(eps1**2-eps1*eps_xy)/(eps1**2 -eps_z*eps_xy)) #Calculate k_x
    print( "k_x " + str(k_x))
    print("Propagation Length: " + str(1/k_x.imag) + '10^-5 cm')
    k_1z = np.emath.sqrt(eps1*(omega2/c)**2 - k_x**2)
    k_2z = np.emath.sqrt(eps_xy*(omega2/c)**2 - (eps_xy/eps_z)*k_x**2)
    if (k_1z.imag) < 0:
        k_1z = -1 * k_1z
    if (k_2z.imag) < 0:
        k_2z = -1 * k_2z
    print("k_1z " + str(k_1z))
    print("k_2z " + str(k_2z))
    skin_depth1 = np.abs((1/k_1z.imag))
    print("Skin Depth Dielectric " + str(skin_depth1))
    skin_depth2 = np.abs((1/k_2z.imag))
    print("Skin Depth Metal " + str(skin_depth2))
    
    
    #Check boundary conditions
    E0 = 1
    E_1z = -(k_x/k_1z) 
    E_2z = (eps_xy/eps_z)*(k_x/k_2z)
    Hy10 = -(omega*eps1/k_1z)*E0
    Hy20 = (omega*eps_xy/k_2z)*E0
    print("Check Magnetic Field Boundary Condition")
    print("Hy1 "+ str(Hy10))
    print("Hy2 "+ str(Hy20))
    
    
    print("Check Displacement Field Boundary Condition")
    print("D_1z " + str(-E_1z * eps1))
    print("D_2z " + str(-E_2z * eps_z))
        
    #Define spatial dimensions of plot
    z = np.linspace(z_lower, z_upper, z_pixels)
    x = np.linspace(0, (4*np.pi/k_x.real), x_pixels)
    
    
    E_x, E_z, Hy, S_x, S_z= calculate_fields_sf(omega, k_x, eps1, eps_xy, eps_z, x, z) #calculate fields
    
    #Plot the semi infinite
    f, a = plt.subplots(figsize= (10, 6))        
    plt.plot(E/c, E, color = 'grey', linestyle = 'dashed', label = "Light line")
    plt.plot(E, omega2*np.ones_like(E), color = 'red', linestyle = 'dashed')
    k_val = anisotropic_semi_infinite(E, dielectric1, dielectric_xy, dielectric_z)
    plt.plot(k_val, E, color = 'black', 
            label = 'Re($k_x$)')
    plt.plot(k_val.imag, E, color = 'grey', 
            label = 'Im($k_x$)')
    plt.axvline(x = k_x, color = 'red', linestyle = 'dashed')
    plt.plot(k_x, omega2, 'o', color = 'red', label = 'Point Analyzed')
    plt.xlabel(r'Re($k_x$) 10$^5$ cm$^{-1}$')
    plt.ylabel("Energy(eV)")
    plt.legend()
    plt.show()

    #Make arrays for vector arrow placement
    x2 = np.linspace(x[0],x[-1], 35)
    z_a = np.linspace(z[0], 0 + spacing, 11)
    z_b = np.linspace(0- spacing, z[-1], 11)
    z2 = np.concatenate((z_a, z_b))
    
    #Re-calculate fields with shape of array for vector arrows
    E_x2, E_z2, Hy2, S_x2, S_z2= calculate_fields_sf(omega, k_x, eps1, eps_xy, eps_z, x2, z2) 
    
    #Make all arrows equal length by normalization 
    e_mag = np.zeros_like(E_z2)
    for i in np.arange(len(E_z2)):
        for j in np.arange(len(E_z2[0])):
            e_mag[i][j]= np.abs(np.emath.sqrt(E_z2[i][j].real**2 + E_x2[i][j].real**2))
    
    if equal_mag == True:
        E_z2 = E_z2/e_mag
        E_x2 = E_x2/e_mag
        
        
    f, a = plt.subplots(figsize = (10, 6), dpi = 600)
    X, Z = np.meshgrid(x2, z2)
    a.set_ylabel('Z ($10^{-5}$ cm)', fontsize = 15)
    a.set_xlabel('X ($10^{-5}$ cm)',fontsize = 15)
    plt.plot(x, np.ones_like(x)* 0, linestyle = 'dashed', color = 'black')
    im = plt.imshow(E_z.real, cmap = cm.bwr, aspect='auto'
           ,extent=[x[0],x[-1],z[0],z[-1]],vmin=None, vmax=None, origin = 'lower')
    
    plt.quiver(X, Z, E_x2.real, -1* E_z2.real, units = 'xy', angles = 'uv', 
               pivot = 'mid', width = 0.1, scale = arrow_scaleE)
    cbar = plt.colorbar(im)
    cbar.set_label('Re $E_z$', rotation = 270)
    if fnameE != None:  #save filename
        plt.savefig(fnameE, bbox_inches = 'tight', dpi = 'figure')
    plt.show()
    
    #Poynting Vector Plot
    f, a = plt.subplots(figsize = (10, 6), dpi = 600)
    X, Z = np.meshgrid(x2, z2)
    a.set_ylabel('Z ($10^{-5}$ cm)', fontsize = 15)
    a.set_xlabel('X ($10^{-5}$ cm)',fontsize = 15)
    plt.plot(x, np.ones_like(x)* 0, linestyle = 'dashed', color = 'black')
    s_mag = np.abs(np.emath.sqrt(S_z.real**2 + S_x.real**2))
    im = plt.imshow(s_mag, cmap = cm.Greens, aspect='auto'
           ,extent=[x[0],x[-1],z[0],z[-1]],vmin=None, vmax=None, origin = 'lower')
    
    plt.quiver(X, Z, S_x2, -1*S_z2, units = 'xy', scale = arrow_scaleS)
    cbar = plt.colorbar(im)
    cbar.set_label('Re $|S|$', rotation = 270)
    if fnameS != None: #save filename
        plt.savefig(fnameS, bbox_inches = 'tight', dpi = 'figure')
    plt.show()

    


In [219]:
def plot_fields(omega, d, E, dielectric1, dielectric_xy, 
                dielectric_z, dielectric3, fnameE = None, fnameS = None, k_min = 1, 
                k_max = 6, k_im_min = 0, k_im_max = 5, vmin0 = 0, vmax0 = 7,
                z_pixels = 150, x_pixels = 150, spacing = 0.05, 
                z_lower = 0.5, z_upper = -0.5, arrow_width = 0.01, arrow_scaleE = 2, arrow_scaleS = 2, complex_k = False, 
                equal_mag = False):
    "Analytical Thin Film Model Electromagnetic Field Plots"
    '''
    omega - Single Energy point(eV)
    d - thickness ( units: 10^5 cm or 100 nm or 0.1 um)
    params - list of parameters in the order in which they appear on the 
    dielectric1 - dielectric of medium 1
    dielectric_xy - in-plane dielectric of medium 2
    dielectric_z - out-of-plane dielectric of medium 2
    dielectric3 - dielectric of medium 3
    fnameE - filename for electric field plt
    fnameS - filename for Poynting vector plot
    solve_rp - Solve for point using reflection. Otherwise find point with best dispersion method
    k_min - minimum k value (units: omega/c) to look for solutions
    k_max - maximum k value (units: omega/c) to look for solutions
    k_im_max - maximum imaginary k value (absolute) to look for solutions
    auto_scale - Boolean. If true autoscales the Im(rp) colormap
    vmin0/vmax0 - manual data range of Im(rp) colormap if auto_scale = False
    z_pixels - number of z points to evaluate field at
    x_pixels - number of x points to evaluate field at
    auto_z - Boolean. Automatic assignment of upper/lower vertical coordinates on graph
    z_lower - lower limit of z (positive is down!)
    z_upper - upper limit of z (positive is down!)
    arrow_width - width of arrow shaft in electric field vector plot
    arrow_scaleE/S - inverse scaling for arrows in vector plots electric field/poynting vector
    complex_k - plot heat map of Im(r_p) over the complex plane in k-space 
    
    
    '''
    c= 1.97
    # Need to assign each parameter to correct dielectric function
    
    ind_w = Get_i(E, omega)
    omega2 = E[ind_w]  #assign omega2 to value in E array closest to desired value 
    eps1 = dielectric1[ind_w]
    eps_xy = dielectric_xy[ind_w]
    eps_z = dielectric_z[ind_w]
    eps3 = dielectric3[ind_w]
        
    
    k_x = solve_point_disp(omega2, d, eps1, eps_xy, eps_z, eps3, k_min, k_max, k_im_min = k_im_min, k_im_max = k_im_max)
    
    if k_x.real < 0 or k_x.imag < 0:
        k_x = -1 * k_x   #imaginary k_x must be positive for decay to be handled correctly
    print("k_x "+ str(k_x))
    
    k_1z = np.emath.sqrt(eps1*(omega2/c)**2 - k_x**2)
    k_2z = np.emath.sqrt(eps_xy*(omega2/c)**2 - (eps_xy/eps_z)*k_x**2)
    k_3z = np.emath.sqrt(eps3*(omega2/c)**2 - k_x**2)
    if k_1z.imag < 0:   #Imaginary k_1z positive
        k_1z = -1 * k_1z
    if k_1z.imag == 0:
        k_1z = np.abs(k_1z)
    if k_2z.imag < 0:  #imaginary k_2z positive
        k_2z = 1 * k_2z
    if k_2z.imag == 0:
        k_2z = np.abs(k_2z)
    k_3z = np.emath.sqrt(eps3*(omega2/c)**2 - k_x**2)
    if k_3z.imag < 0: #Imaginary k_3z positive
        k_3z = -1 * k_3z
    if k_3z.imag == 0:
        k_3z = np.abs(k_3z)
    
    skin_depth1 = np.abs((1/k_1z.imag))   #Calculate skin depth (z value where fields drops off to 1/e)
    skin_depth3 = np.abs((1/k_3z.imag))
    print("k_1z " + str(k_1z))
    print("k_2z " + str(k_2z))
    print("k_3z " + str(k_3z))
    
    print("Skin Depth Medium 1: " + str(skin_depth1))
    print("Skin Depth Medium 3: " + str(skin_depth3))
    
    prop_len = np.abs((1/k_x.imag))  #In-plane propagation length
    print("Propagation Length " + str(prop_len))
    
    z = np.linspace(z_lower, z_upper, z_pixels)
    
             
    #Coefficients for fields that solve Maxwell equations, fit dispersion  
    alpha = (1j) *k_2z * d
    r_12 = (eps1*k_2z - eps_xy*k_1z)/(eps1*k_2z + eps_xy*k_1z)
    r_23 = (eps_xy*k_3z - eps3*k_2z)/(eps_xy*k_3z + eps3 * k_2z)
    t_23 = 1 + r_23
    t_12 = 1 + r_12
    
    print("Dispersion quality " + str(1 + r_12*r_23 * np.exp(2*alpha))) #Check quality of solution (should be 0)
    x = np.linspace(0, (np.pi*2/k_x.real), x_pixels)  #x values out to two wavelengths of plasmon
    
    
    
    E_3x = 1
    E_2x_plus0 = E_3x/t_23
    E_2x_minus0 = E_3x*r_23/t_23
    
    
    E_2x_plus = E_2x_plus0 * np.exp(-alpha)
    E_2x_minus = E_2x_minus0 * np.exp(alpha)
    E_1x = (1/t_12)*(r_12*E_2x_plus + E_2x_minus)
    
    
    
    
    # Check Boundary Conditions
    H_y10 = -(omega2/k_1z) * eps1*E_1x
    H_y20 = (omega2/k_2z) * eps_xy * (E_2x_plus - E_2x_minus)
    if H_y10 != H_y20:
        print("Check magnetic boundary condition at z = 0")
        print(H_y10)
        print(H_y20)
        
    H_y2d = (omega2/k_2z) * eps_xy * (E_2x_plus0 - E_2x_minus0)
    H_y3d = (omega2/k_3z)* eps3 * E_3x
    if H_y2d != H_y3d:
        print("Check magnetic boundary condition at z = d")
        print(H_y2d)
        print(H_y3d)
  

    E_x, E_z, Hy, S_x, S_z = calculate_fields(omega2, k_x, d, eps1, eps_xy, eps_z, eps3, x, z)
    
    #Plot reflection coefficient to get the dispersion  
    k1 = np.linspace(0, k_x.real + 1, 200)
    f,a=plt.subplots(1,1,dpi=200,figsize=(10,6))
    a.set_title('Plasmonic Dispersion', fontsize = 20)
    rp = reflection_coefficient(E, k1, d, dielectric1, 
                               dielectric_xy, dielectric_z,
                               dielectric3)
    im = a.imshow(rp, cmap = cm.plasma, aspect='auto'
           ,extent=[k1[0],k1[-1],E[0],E[-1]],vmin=vmin0,vmax=vmax0, origin = 'lower')
    cbar = plt.colorbar(im)
    cbar.set_label('Im $r_p$')
    if k_max*omega >= E[-1]: a.set_ylim(0, E[-1])
    else:  a.set_ylim(0, k_max*omega)
    a.set_xlim(0, k_x.real + 1)
    a.set_ylabel('Energy (eV)', fontsize = 15)
    a.set_xlabel('Re($k_x$)  $\mu$m$^{-1}$', fontsize = 15)
    
    # Plot point being analyzed
    plt.plot(E/1.97, E, color = 'white', linestyle = 'dashed')
   
    plt.plot(k1, omega2*np.ones_like(k1), color = 'red', linestyle = 'dashed')
    plt.axvline(x = k_x, color = 'red', linestyle = 'dashed')
    plt.plot(k_x, omega2, 'o', color = 'red', label = 'Point Analyzed')

    plt.xlabel(r'Re($k_x$)  $\mu$m$^{-1}$')
    plt.ylabel("Energy(eV)")
    plt.legend()
    plt.show()
    
    
    #Plot Complex k-space
    if complex_k:
        k_im = np.linspace(0, k_im_max, 300)
        k_re = np.linspace(0, k_max * (omega/c) + 1, 250)
        kmap = plot_complex_k(omega, k_re, k_im, d, eps1, eps_xy, eps_z, eps3)
        f, a = plt.subplots(figsize = (10, 6))
        a.set_title("k-Space Map at " + str(omega) + " eV")
        im = a.imshow(kmap, cmap = cm.plasma, aspect='auto'
           ,extent=[k_re[0],k_re[-1],k_im[0],k_im[-1]],vmin=vmin0,vmax= 0.3 * np.nanmax(kmap), origin = 'lower')
        cbar = plt.colorbar(im)
        cbar.set_label('Im $r_p$')
        a.set_ylim(0, k_im[-1])
        a.set_xlim(0, k_re[-1])
        a.set_ylabel('Im($k_x$)  $\mu$m$^{-1}$', fontsize = 15)
        a.set_xlabel('Re($k_x$)  $\mu$m$^{-1}$', fontsize = 15)
        plt.axvline(x = omega2/c, color = 'white', linestyle = 'dashed', 
                    label = 'light line ($k_0 = $' + str(round(omega/c, 2)) + ")" )
        plt.axvline(x = k_min * (omega2/c), color = 'lime', linestyle = 'dashed', label = 'k_min')
        plt.axvline(x = k_max * (omega2/c), color = 'lime', linestyle = 'dashed', label = 'k_max')
        plt.plot(k_x.real, k_x.imag, 'x', color = 'lime')
        plt.legend()
        plt.show()
    
    
    #Construct arrays for vector arrays coordinates
    x2 = np.linspace(x[0],x[-1], 30)
    z_a = np.linspace(z[0], d + spacing, 10)
    z_b = np.linspace(d - spacing , 0 + spacing, int(d/0.02))
    z_c = np.linspace(0 - spacing, z[-1], 10)
    z2 = np.concatenate((z_a, z_b, z_c))
    
    
    E_x2, E_z2, Hy2, S_x2, S_z2 = calculate_fields(omega, k_x, d, eps1, eps_xy, eps_z, eps3, x2, z2)
    
    e_mag = np.zeros_like(E_z2)
    for i in np.arange(len(E_z2)):
        for j in np.arange(len(E_z2[0])):
            e_mag[i][j]= np.abs(np.emath.sqrt(E_z2[i][j].real**2 + E_x2[i][j].real**2))
    
    if equal_mag == True:
        E_z2 = E_z2/e_mag
        E_x2 = E_x2/e_mag
    
    #Plot Electric Field Vector Plot
    f, a = plt.subplots(figsize = (10, 6), dpi = 600)
    X, Z = np.meshgrid(x2, z2)
    a.set_ylabel('Z ($10^5$ cm$^{-1}$)', fontsize = 15)
    a.set_xlabel('X ($10^5$ cm$^{-1}$',fontsize = 15)
    plt.plot(x, np.ones_like(x)* 0, linestyle = 'dashed', color = 'black')
    plt.plot(x, np.ones_like(x)* d, linestyle = 'dashed', color = 'black')
    im = plt.imshow(E_z.real, cmap = cm.bwr, aspect='auto'
           ,extent=[x[0],x[-1],z[0],z[-1]],vmin=None, vmax=None, origin = 'lower')
    plt.quiver(X, Z,  E_x2.real, -1*E_z2.real, units = 'xy', pivot = 'mid', width = arrow_width, scale = arrow_scaleE)
    cbar = plt.colorbar(im, pad = 0.01)
    cbar.set_label('Re $E_z$')
    
    if fnameE != None:  #save filename
        plt.savefig(fnameE, bbox_inches = 'tight', dpi = 'figure')
    plt.show()
    plt.show()
    
    
    #Plot Poynting Vector
    f, a = plt.subplots(figsize = (10, 6), dpi = 600)
    X, Z = np.meshgrid(x2, z2)
    a.set_ylabel('Z ($10^5$ cm$^{-1}$)', fontsize = 15)
    a.set_xlabel('X ($10^5$ cm$^{-1}$)',fontsize = 15)
    plt.plot(x, np.ones_like(x)* d, linestyle = 'dashed', color = 'black')
    plt.plot(x, np.ones_like(x)* 0, linestyle = 'dashed', color = 'black')
    s_mag = np.abs(np.emath.sqrt(S_z.real**2 + S_x.real**2))
    im = plt.imshow(s_mag, cmap = cm.Greens, aspect='auto'
           ,extent=[x[0], x[-1], z[0],z[-1]],vmin=0, vmax= 0.8 * np.nanmax(s_mag), origin = 'lower')
    plt.quiver(X, Z, S_x2.real, -1*S_z2.real, units = 'xy', pivot = 'mid', scale = arrow_scaleS)
    cbar = plt.colorbar(im, pad = 0.01)
    cbar.set_label('$|S|$')
    if fnameS != None:  #save filename
        plt.savefig(fnameS, bbox_inches = 'tight', dpi = 'figure')
    plt.show()
    

In [218]:
def plot_fields_4l(omega, d1, d2, E, dielectric1, dielectric_xy, 
                dielectric_z, dielectric3, dielectric4, fnameE = None, fnameS = None, k_min = 1, 
                k_max = 6, k_im_min = 0, k_im_max = 5, vmin0 = 0, vmax0 = 7, 
                z_pixels = 150, x_pixels = 150, 
                z_lower = 0.5, z_upper = -0.5, arrow_width = 0.01, arrow_scaleE = 2, arrow_scaleS = 2, complex_k = False, 
                complexk_max = 0.7, equal_mag = False):
    "Analytical Thin Film Model Electromagnetic Field Plots"
    '''
    omega - Single Energy point(eV)
    d1/d2 - thickness ( units: 10^5 cm or 100 nm or 0.1 um) 
    dielectric1 - dielectric of medium 1
    dielectric_xy - in-plane dielectric of medium 2
    dielectric_z - out-of-plane dielectric of medium 2
    dielectric3 - dielectric of medium 3
    dielectric4 -dielectric of medium 4
    solve_rp - Solve for point using reflection. Otherwise find point with best dispersion method
    k_min - minimum k value (units: omega/c) to look for solutions
    k_max - maximum k value (units: omega/c) to look for solutions
    k_im_max - maximum imaginary k value (absolute) to look for solutions
    vmin0/vmax0 - manual data range of Im(rp) colormap if auto_scale = False
    complexk_max - intensity (in 1/max(rp)) for complex_k colormap
    z_pixels - number of z points to evaluate field at
    x_pixels - number of x points to evaluate field at
    z_lower - the upper limit on the z-axis to display if auto_z = False
    z_upper - the lower limit on the z-axis to display if auto_z = False
    arrow_width = with of arrows for electric field vector plot
    arrow_scaleE/S - inverse scaling for arrows in vector plots electric field/poynting vector
    complex_k - plot heat map of Im(r_p) over the complex plane in k-space 

    '''
    c= 1.97
    # Need to assign each parameter to correct dielectric function
    
    ind_w = Get_i(E, omega)
    omega2 = E[ind_w]
    eps1 = dielectric1[ind_w]
    eps_xy = dielectric_xy[ind_w]
    eps_z = dielectric_z[ind_w]
    eps3 = dielectric3[ind_w]
    eps4 = dielectric4[ind_w]  
    k_x = solve_point_disp4(E[ind_w], d1, d2, eps1, eps_xy, eps_z, eps3, eps4, k_min, k_max, 
                                  k_im_min = k_im_min, 
                                  k_im_max = k_im_max)
    
    if k_x.real < 0 or k_x.imag < 0:
        k_x = -1 * k_x   #imaginary k_x must be positive for decay to be handled correctly
    print("k_x "+ str(k_x))
    print("k_x/k0 " + str( (k_x*c/omega2).real))
    
    k_1z = np.emath.sqrt(eps1*(omega2/c)**2 - k_x**2)
    k_2z = np.emath.sqrt(eps_xy*(omega2/c)**2 - (eps_xy/eps_z)*k_x**2)
    k_3z = np.emath.sqrt(eps3*(omega2/c)**2 - k_x**2)
    k_4z = np.emath.sqrt(eps4*(omega2/c)**2 - k_x**2)
    if k_1z.imag < 0:   #Imaginary k_1z positive
        k_1z = -1 * k_1z
    if k_2z.imag < 0:  #imaginary k_2z positive
        k_2z = 1 * k_2z
    if k_2z.imag == 0:
        k_2z = np.abs(k_2z)
    #k_3z = np.emath.sqrt(eps3*(omega/c)**2 - k_x**2)
    if k_3z.imag < 0: #Imaginary k_3z positive
        k_3z = -1 * k_3z
    if k_4z.imag < 0: #Imaginary k_3z positive
        k_4z = -1 * k_4z

    
    skin_depth1 = np.abs((1/k_1z.imag))   #Calculate skin depth (z value where fields drops off to 1/e)
    skin_depth3 = np.abs((1/k_3z.imag))
    skin_depth4 = np.abs((1/k_4z.imag))
    print("k_1z " + str(k_1z))
    print("k_2z " + str(k_2z))
    print("k_3z " + str(k_3z))
    print("k_4z " + str(k_4z))
    
    print("Skin Depth Medium 1: " + str(skin_depth1))
    print("Skin Depth Medium 3: " + str(skin_depth3))
    print("Skin Depth Medium 4: " + str(skin_depth4))
    
    prop_len = np.abs((1/k_x.imag))  #In-plane propagation length
    print("Propagation Length " + str(prop_len) + " cm 10^-5")
    
    z = np.linspace(z_lower, z_upper, z_pixels)
    
    #Coefficients for fields that solve Maxwell equations, fit dispersion  
    
    alpha2 = (1j) * k_2z * d1
    alpha3 = (1j) * k_3z * d2
    r_12 =  (eps1*k_2z - eps_xy*k_1z)/(eps1*k_2z + eps_xy*k_1z)
    r_23 = (eps_xy*k_3z - eps3*k_2z)/(eps_xy*k_3z + eps3 * k_2z)
    r_34 =(eps3*k_4z - eps4*k_3z )/(eps3*k_4z + eps4*k_3z)
    disp = 1 + r_23*r_34 * np.exp(2*alpha3) + r_12*r_23*np.exp(2*alpha2) + r_12*r_34*np.exp(2*alpha2 + 2*alpha3)
    print("Dispersion quality " + str(disp))
    t_23 = 1 + r_23
    t_12 = 1 + r_12
    t_34 = 1 + r_34
    D4 = 1
    E_4x = 1
    E_3x_plus = E_4x/t_34
    E_3x_minus = r_34 * E_4x/t_34
    print("check electric field boundary at z = d1 + d2")
    print(E_4x)
    print(str(E_3x_plus + E_3x_minus))
    
    print("check magnetic field boundary at z = d1 + d2")
    print(E_4x * eps4/k_4z)
    print(str((eps3/k_3z)* (E_3x_plus - E_3x_minus)))
    
    
    E_2x_plus = (1/t_23) * (E_3x_plus*np.exp(-alpha3) + r_23 * E_3x_minus * np.exp(alpha3))
    E_2x_minus = (1/t_23) * (r_23*E_3x_plus*np.exp(-alpha3) + E_3x_minus * np.exp(alpha3))
    print("check electric field boundary at z = d1")
    print(str(E_3x_plus*np.exp(-alpha3) + E_3x_minus * np.exp(alpha3)))
    print(str(E_2x_plus + E_2x_minus))
    
    print("check magnetic field boundary at z = d1")
    print(eps3*(E_3x_plus*np.exp(-alpha3) - E_3x_minus * np.exp(alpha3))/k_3z)
    print(str((eps_xy/k_2z)* (E_2x_plus - E_2x_minus)))
    
    E_1x = (1/t_12) * (r_12*E_2x_plus*np.exp(-alpha2) + E_2x_minus * np.exp(alpha2))
    print("check electric field boundary at z = 0")
    print(str(E_2x_plus*np.exp(-alpha2) + E_2x_minus * np.exp(alpha2)))
    print(E_1x)
    
    print("check magnetic field boundary at z = 0")
    print( str((E_2x_plus*np.exp(-alpha2) - E_2x_minus * np.exp(alpha2))* eps_xy/k_2z))
    print(str(-(eps1/k_1z)*E_1x))
    
    print("check displacement at z = 0")
    print(-eps_z*(eps_xy/eps_z)*(k_x/k_2z)*(E_2x_plus *np.exp(-alpha2) - E_2x_minus* np.exp(alpha2)))
    print(eps1*(k_x/k_1z)*E_1x)

    #print("Dispersion quality 2" + str(1 + r_23*r_34 * np.exp(2*alpha3) + r_12*r_23*np.exp(2*alpha2) + r_12*r_34*np.exp(2*alpha2 + 2*alpha3)))
    x = np.linspace(0, (np.pi*2/k_x.real), x_pixels)  #x values out to two wavelengths of plasmon
    
    
    
    E_x, E_z, Hy, S_x, S_z = calculate_fields_4l(omega2, k_x, d1,d2, eps1, eps_xy, eps_z, eps3, eps4, x, z)
    
    #Plot reflection coefficient to get the dispersion  
    k1 = np.linspace(0, k_x.real + 1, 200)
    f,a=plt.subplots(1,1,dpi=200,figsize=(10,6))
    a.set_title('Plasmonic Dispersion', fontsize = 20)
    rp = reflection_coefficient_4layer(E, k1, d1, d2, dielectric1, 
                               dielectric_xy, dielectric_z,
                               dielectric3, dielectric4)
    im = a.imshow(rp, cmap = cm.plasma, aspect='auto'
           ,extent=[k1[0],k1[-1],E[0],E[-1]],vmin=vmin0,vmax=vmax0, origin = 'lower')
    cbar = plt.colorbar(im)
    cbar.set_label('Im $r_p$')
    if k_max*omega2 >= E[-1]: a.set_ylim(0, E[-1])
    else:  a.set_ylim(0, k_max*omega2)
    a.set_xlim(0, k_x.real + 1)
    a.set_ylabel('Energy (eV)', fontsize = 15)
    a.set_xlabel('Re($k_x$) 10$^5$ cm$^{-1}$', fontsize = 15)
    
    
    # Plot point being analyzed
    plt.plot(E/1.97, E, color = 'white', linestyle = 'dashed')
   
    plt.plot(k1, omega2*np.ones_like(k1), color = 'red', linestyle = 'dashed')
    plt.axvline(x = k_x, color = 'red', linestyle = 'dashed')
    plt.plot(k_x, omega2, 'o', color = 'red', label = 'Point Analyzed')

    plt.xlabel(r'Re($k_x$) 10$^5$ cm$^{-1}$')
    plt.ylabel("Energy(eV)")
    plt.legend()
    plt.show()
    
    
    #Plot Complex k-space
    if complex_k:
        k_im = np.linspace(0, k_im_max, 300)
        k_re = np.linspace(0, k_max * (omega2/c) + 1, 250)
        kmap = plot_complex_k4(omega2, k_re, k_im, d1, d2, eps1, eps_xy, eps_z, eps3, eps4)
        f, a = plt.subplots(figsize = (10, 6))
        a.set_title("k-Space Map at " + str(omega) + " eV")
        im = a.imshow(kmap, cmap = cm.plasma, aspect='auto'
           ,extent=[k_re[0],k_re[-1],k_im[0],k_im[-1]],vmin=vmin0,vmax= complexk_max * np.nanmax(kmap), 
                      origin = 'lower')
        cbar = plt.colorbar(im)
        cbar.set_label('Im $r_p$')
        a.set_ylim(0, k_im[-1])
        a.set_xlim(0, k_re[-1])
        a.set_ylabel('Im($k_x$) 10$^5$ cm$^{-1}$', fontsize = 15)
        a.set_xlabel('Re($k_x$) 10$^5$ cm$^{-1}$', fontsize = 15)
        plt.axvline(x = omega2/c, color = 'white', linestyle = 'dashed', 
                    label = 'light line ($k_x = $' + str(round(omega2/c, 2)) + ")" )
        plt.axvline(x = k_min * (omega2/c), color = 'lime', linestyle = 'dashed', label = 'k_min')
        plt.axvline(x = k_max * (omega2/c), color = 'lime', linestyle = 'dashed', label = 'k_max')
        plt.plot(k_x.real, k_x.imag, 'x', color = 'lime')
        plt.legend()
        plt.show()
    
  
    x2 = np.linspace(x[0],x[-1], 35)
    z2 = np.linspace(z_lower,z_upper, 35)
    
    E_x2, E_z2, Hy2, S_x2, S_z2 = calculate_fields_4l(omega2, k_x, d1,d2, eps1, eps_xy, eps_z, 
                                                      eps3, eps4, x2, z2)
    e_mag = np.zeros_like(E_z2)
    for i in np.arange(len(E_z2)):
        for j in np.arange(len(E_z2[0])):
            e_mag[i][j]= np.abs(np.emath.sqrt(E_z2[i][j].real**2 + E_x2[i][j].real**2))
    
    if equal_mag == True:
        E_z2 = E_z2/e_mag
        E_x2 = E_x2/e_mag
        
        
        
    #Electric Field Vector Plot   
    f, a = plt.subplots(figsize = (10, 6), dpi = 300)
    X, Z = np.meshgrid(x2, z2)
    a.set_ylabel('Z ($10^{-5}$ cm)', fontsize = 15)
    a.set_xlabel('X ($10^{-5}$ cm)',fontsize = 15)
    plt.plot(x, np.ones_like(x)* 0, linestyle = 'dashed', color = 'black')
    plt.plot(x, np.ones_like(x)* d1, linestyle = 'dashed', color = 'black')
    plt.plot(x, np.ones_like(x)* (d1+d2), linestyle = 'dashed', color = 'black')
    im = plt.imshow(E_z.real, cmap = cm.bwr, aspect='auto'
           ,extent=[x[0],x[-1],z[0],z[-1]],vmin=None, vmax=None, origin = 'lower')
    plt.quiver(X, Z,  E_x2.real, -1*E_z2.real, units = 'xy', width = arrow_width, pivot = 'mid', scale = arrow_scaleE)
    cbar = plt.colorbar(im, pad = 0.01)
    cbar.set_label('Re $E_z$')
   
    if fnameE != None: #save filename
        plt.savefig(fnameE, bbox_inches = 'tight', dpi = 'figure')
    plt.show()
    
    #Poynting Vector Plot
    f, a = plt.subplots(figsize = (10, 6), dpi = 300)
    X, Z = np.meshgrid(x2, z2)
    a.set_ylabel('Z ($10^{-5}$ cm)', fontsize = 15)
    a.set_xlabel('X ($10^{-5}$ cm)',fontsize = 15)
    plt.plot(x, np.ones_like(x)* d1, linestyle = 'dashed', color = 'black')
    plt.plot(x, np.ones_like(x)* (d1+d2), linestyle = 'dashed', color = 'black')
    plt.plot(x, np.ones_like(x)* 0, linestyle = 'dashed', color = 'black')
    s_mag = np.abs(np.emath.sqrt(S_z.real**2 + S_x.real**2))
    im = plt.imshow(s_mag, cmap = cm.Greens, aspect='auto'
           ,extent=[x[0],x[-1],z[0],z[-1]],vmin=None, vmax=None, origin = 'lower')
    plt.quiver(X, Z, S_x2, -1*S_z2, units = 'xy',pivot = 'mid', scale = arrow_scaleS)
    cbar = plt.colorbar(im, pad = 0.01)
    cbar.set_label('$|S|$')
    if fnameS != None: #save filename
        plt.savefig(fnameS, bbox_inches = 'tight', dpi = 'figure')
    plt.show()

    
    
    
    return k_x, E_x, E_z, Hy, S_x, S_z